In [ ]:
import json
# with open("/scr/ig_pipeline/artifacts/pipeline/object_inventory.json", "r") as f:
#     providers = json.load(f)["providers"]

In [16]:
import glob, pathlib, collections
users = collections.defaultdict(set)
shows_up_as = collections.defaultdict(set)
for dirn in glob.glob("D:/ig_pipeline/cad/*/*"):
    fn = pathlib.Path(dirn) / "artifacts/object_list.json"
    if not fn.exists():
        print(fn, "is missing")
        continue
    with open(fn, "r") as f:
        target = "/".join(fn.parts[-4:-2])
        object_list = json.load(f)["needed_objects"]
        for obj in object_list:
            cat, mdl = obj.split("-")
            users[mdl].add(target)
            shows_up_as[mdl].add(cat)

In [17]:
used_keys = {x.split("-")[-1] for x in users}
assert all(len(x) == 6 for x in used_keys)

In [18]:
# What objects show up as multiple things?
for k, v in shows_up_as.items():
    if len(v) > 1:
        print(k, v)

In [19]:
import csv
with open("D:/ig_pipeline/metadata/object_renames.csv", "r") as f:
    reader = csv.DictReader(f)
    renames = {}
    for row in reader:
        obj_id = row["ID (auto)"]
        old_cat = row["Original category (auto)"]
        new_cat = row["New Category"]
        
        if obj_id not in used_keys:
            print(f"{obj_id} not provided")
            continue
            
        if obj_id in renames:
            print(f"{obj_id} renamed twice")
        
        if obj_id in shows_up_as:
            if shows_up_as[obj_id] != {old_cat} and shows_up_as[obj_id] != {new_cat}:
                print(f"{old_cat}-{obj_id} shows up with unexpected categories {shows_up_as[obj_id]} in inventory. Suggested rename is to {new_cat}")
                continue

            if shows_up_as[obj_id] != {new_cat}:
                print(f"{obj_id} will be renamed to {new_cat} but shows up as {shows_up_as[obj_id]} in inventory.")

        renames[obj_id] = new_cat

In [20]:
renames

{'pttzdw': 'bottle_of_apple_juice',
 'xvrbdy': 'bottle_of_apple_juice',
 'zjzgjy': 'box_of_apple_juice',
 'fabztm': 'webcam',
 'fckfij': 'pack_of_bread',
 'sfupqo': 'pack_of_bread',
 'wjbnfu': 'pack_of_bread',
 'yehtiv': 'pack_of_bread',
 'bbyvsc': 'bag_of_cookies',
 'ikivgk': 'bag_of_cookies',
 'ksjtde': 'bag_of_cookies',
 'vafomx': 'bag_of_cookies',
 'ikbsox': 'bottle_of_barbecue_sauce',
 'nkqvex': 'bottle_of_barbecue_sauce',
 'rzevkb': 'bottle_of_barbecue_sauce',
 'lzrdiz': 'box_of_beer',
 'ubvfih': 'box_of_beer',
 'bwaboq': 'bottle_of_beer',
 'crfcwo': 'bottle_of_beer',
 'dcwvkg': 'bottle_of_beer',
 'dqfsgv': 'bottle_of_beer',
 'eicnxj': 'bottle_of_beer',
 'fcnrqt': 'bottle_of_beer',
 'fgzjnb': 'bottle_of_beer',
 'gxxbhh': 'bottle_of_beer',
 'hauvsg': 'bottle_of_beer',
 'ikgezm': 'bottle_of_beer',
 'jssuog': 'bottle_of_beer',
 'jtgyoo': 'bottle_of_beer',
 'jxhtdl': 'bottle_of_beer',
 'kqskmv': 'bottle_of_beer',
 'meqliv': 'bottle_of_beer',
 'mhzpkh': 'bottle_of_beer',
 'miiijl': 'b

: 

In [ ]:
with open("D:/ig_pipeline/metadata/deletion_queue.csv", "r") as f:
    reader = csv.DictReader(f)
    for row in reader:
        obj_name = row["Object"]
        cat, obj_id = obj_name.split("-")[:2]
        obj_id = obj_id.strip()
        assert len(obj_id) == 6, obj_id
        
        if obj_id not in used_keys:
            print(f"{obj_id} not provided")
            continue
            
        if obj_id in renames:
            print(f"{obj_id} renamed and deleted")
        
        if f"{cat}-{obj_id}" not in users:
            found_cat, = [x.split("-")[0] for x in users if x.split("-")[-1] == obj_id]
            print(f"{cat}-{obj_id} shows up with unexpected category {found_cat} in inventory.")